In [1]:
from bs4 import BeautifulSoup as bs

import requests
import re


import pandas as pd
import numpy as np
import pyodbc


import time
import random

import os

In [2]:
# this is grabing the Cardinals html web page
#url = 'https://www.pro-football-reference.com/teams/sfo/2022.htm'

# Table Function

In [17]:
# use this function before creating tables
def tables_html(url:str):
    html = requests.get(url).text
    
    soup = bs(html, 'html.parser')
    
    tables = soup.find_all(class_='table_wrapper')
    
    return tables

# DF Columns!

In [4]:
def webscrape_columns (tables:str, table_num:int):

    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')
    
    
    #find thead tag to isolate col names
    table_html = table_html.find_all('thead')[0]

    # find class_ = 'poptip' to isolate col names
    col_th_tag = table_html.find_all(class_='poptip')

    # get how many column names there are
    num_col_passing = len(col_th_tag)

    # list of column names
    columns = []

    for i in range(0, num_col_passing):
        try:
            data = col_th_tag[i].contents[0]
            columns.append(data)
        except:
            columns.append('n/a')   
            
    return columns

# DF Rows!

## Row

In [5]:
def webscrape_index(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
       
    #get index
    num_index = len(rows_tbody.find_all('th'))
    
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
    return index

In [6]:
def webscrape_rows(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
    

    rows_tr = rows_tbody.find_all('tr')
    
    num_index = len(rows_tbody.find_all('th'))
    num_rows = len(rows_tr[0].find_all('td'))
    
    
    row_data = []

    for i in range(0, num_index):
        row_td = rows_tr[i].find_all('td')
    
        data = []

        for c in range(0, num_rows):
            
            try:
                if len(row_td[c].find_all('a')) > 0:
            
                    d = row_td[c].find_all('a')[0].contents[0]
                    data.append(d)
                
                else:
                    d = row_td[c].contents[0]
                    data.append(d)
            
            except:
                data.append('n/a')
        

        row_data.append(data)
        
        
        
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
        
    #insert index to each row to be equal to all columns
    for i in range(0, len(index)):
        row_data[i].insert(0, index[i])
        
        
    return row_data

# Fix Column Labels

In [7]:
#fix defense and fumbles table

In [8]:

def fix_col_labels(table_df_li: list, team: str):
    
    
    #Team Stats and Rankings
    col_li = list(table_df_li[0].columns)
    
    col_li[2] = 'Total Yds and TO'
    col_li[10] = 'Passing Yds'
    col_li[16] = 'Russhing Yds'
    col_li[21] = 'Penalty Yds'
    col_li[23] = 'Dr'
    
    table_df_li[0].columns = col_li
    
    
    #Schedule and Game Results
    
        #fix col to names
    try:
        table_df_li[1].columns = ['Week', 'Day', 'Date', 'Time', 'n/a', 'Win/Loss', 'OT', 'Rec', 'Home/Away',
                              'Opp', 'Tm', 'Opp', '1stD', 'TotYd', 'PassY', 'RushY', 'TO', '1stD',
                              'TotYd', 'PassY', 'RushY', 'TO', 'Offense', 'Defense', 'Sp. Tms']

        table_df_li[1].drop(columns=['n/a'], inplace=True)

        
    except:
        table_df_li[1].columns = ['Week', 'Day', 'Date', 'Time', 'Win/Loss', 'OT', 'Rec', 'Home/Away',
                              'Opp', 'Tm', 'Opp', '1stD', 'TotYd', 'PassY', 'RushY', 'TO', '1stD',
                              'TotYd', 'PassY', 'RushY', 'TO', 'Offense', 'Defense', 'Sp. Tms']
    
        # home/away games rows
    table_df_li[1].iloc[:, 7] = ['away' if r == '@' else 'home' for r in table_df_li[1].iloc[:, 7]]
          
        
        
    #Passing
    col_li = list(table_df_li[3].columns)
    
    col_li[10] = 'Yds Gained'
    col_li[20] = 'Yds Lost'

    table_df_li[3].columns = col_li
    
    
    #Rushing and Receiving
    col_li = list(table_df_li[4].columns)
    
    col_li[7] = 'Rushing Yds'
    col_li[8] = 'Rushin TD'
    col_li[9] = 'First Down Rushing'
    col_li[10] = 'Rushing Success'
    col_li[11] = 'Longest Rushing Attempt'
    col_li[13] = 'Rushing Yds / Game'
    col_li[17] = 'Receiving Yds'
    col_li[19] = 'Receiving TD'
    col_li[20] = 'First Down Receiving'
    col_li[21] = 'Receiving Success Rate'
    col_li[22] = 'Longest Reception'
    col_li[24] = 'Receives / Game'
    
    
    table_df_li[4].columns = col_li
    
    #kick and punt returns
    col_li = list(table_df_li[5].columns)
    
    col_li[7] = 'Punt Returns Yds'
    col_li[8] = 'Punt Return TD'
    col_li[9] = 'Longest Punt Return'
    col_li[12] = 'Kick Returns Yds'
    col_li[13] = 'Kick Returns TD'
    col_li[14] = 'Longest Kickoff Return'

    table_df_li[5].columns = col_li
    
    #Kicking
    col_li = list(table_df_li[6].columns)
    
        #0-19
    col_li[6] = 'FGA 0-19'
    col_li[7] = 'FGM 0-19'
    
        #20-29
    col_li[8] = 'FGA 20-29'
    col_li[9] = 'FGM 20-29'
    
        #30-39
    col_li[10] = 'FGA 30-39'
    col_li[11] = 'FGM 30-39'
    
        #40-49
    col_li[12] = 'FGA 40-49'
    col_li[13] = 'FGM 40-49'
    
        #50+
    col_li[14] = 'FGA 50+'
    col_li[15] = 'FGM 50+'

    table_df_li[6].columns = col_li
    
    #defense and fumbles
    col_li = list(table_df_li[8].columns)
    
    col_li[7] = 'Def Interceptions Yds'
    col_li[8] = 'Def Touchdown'
    col_li[14] = 'Fumbles Yds'
    col_li[15] = 'Fumbles Touchdown'

    table_df_li[8].columns = col_li
    
    #touchdown log
    try:
        table_df_li[10].drop(columns=['n/a'], inplace=True)
    
    #opponenet touchdown log
        table_df_li[11].drop(columns=['n/a'], inplace=True)
    except:
        pass
    
    
    for i in range(0, len(table_df_li)):
        num_rows = len(table_df_li[i].values)

        table_df_li[i].insert(1, 'Team', np.repeat(team, num_rows))
    
    return table_df_li



# DF!

In [9]:
#df = pd.DataFrame(index=index, data=row_data, columns=columns)
#df

In [32]:
def webscrape_tables(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
    
    #columns
    #find thead tag to isolate col names
    table_html = table_html.find_all('thead')[0]

    # find class_ = 'poptip' to isolate col names
    col_th_tag = table_html.find_all(class_='poptip')

    # get how many column names there are
    num_col_passing = len(col_th_tag)

    # list of column names
    columns = []

    for i in range(0, num_col_passing):
        try:
            data = col_th_tag[i].contents[0]
            columns.append(data)
        except:
            columns.append('n/a') 
    
    
    ##rows
    
    rows_tr = rows_tbody.find_all('tr')
    
    num_index = len(rows_tbody.find_all('th'))
    num_rows = len(rows_tr[0].find_all('td'))
    
    
    row_data = []

    for i in range(0, num_index):
        row_td = rows_tr[i].find_all('td')
    
        data = []

        for c in range(0, num_rows):
            
            try:
                if len(row_td[c].find_all('a')) > 0:
            
                    d = row_td[c].find_all('a')[0].contents[0]
                    data.append(d)
                
                else:
                    d = row_td[c].contents[0]
                    data.append(d)
            
            except:
                data.append('n/a')
        

        row_data.append(data)
        
        
        
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
        
    #insert index to each row to be equal to all columns
    for i in range(0, len(index)):
        row_data[i].insert(0, index[i])
    
    
    df = pd.DataFrame(index=index, data=row_data, columns=columns)
    
    return df

# Final Save CSV Function

In [39]:
def webscrape_create_dataframes_save_csv(team: str, year: int):    
    
    team_to_url_df = pd.DataFrame(
        data=[
            [
                'crd', #Cardinals
                'atl', #Falcons
                'rav', #Ravens
                'buf', #Buffalos
                'car', #Panthers
                'chi', #Bears
                'cin', #Bengals
                'cle', #Browns
                'dal', #Cowboys
                'den', #Broncos
                'det', #Lions
                'gnb', #Packers
                'htx', #Texans
                'clt', # Colts
                'jax', #Jaguars
                'kan', #Chiefs
                'rai', #Raiders
                'sdg', #Chargers
                'ram', #Rams
                'mia', #Dolphins
                'min', #Vikings
                'nwe', #Patriots
                'nor', #Saints
                'nyg', #Giants
                'nyj', #Jets
                'phi', #Eagles
                'pit', #Steelers
                'sfo', #49ers
                'sea', #Seahawks
                'tam', #Buccaneers
                'oti', #Titans
                'was' #Commanders
            ]
        ], 
        columns=[
            'Cardinals',
            'Falcons',
            'Ravens',
            'Buffalos',
            'Panthers',
            'Bears',
            'Bengals',
            'Browns',
            'Cowboys',
            'Broncos',
            'Lions',
            'Packers',
            'Texans',
            'Colts',
            'Jaguars',
            'Chiefs',
            'Raiders',
            'Chargers',
            'Rams',
            'Dolphins',
            'Vikings',
            'Patriots',
            'Saints',
            'Giants',
            'Jets',
            'Eagles',
            'Steelers',
            '49ers',
            'Seahawks',
            'Buccaneers',
            'Titans',
            'Commanders'
        ])
    
    # string url
    str_url = 'https://www.pro-football-reference.com/teams/{}/{}.htm'

    #get team url ID
    team_url_str = team_to_url_df[team].values[0]
    
    # URL
    url = str_url.format(team_url_str, year)
    
    # request html text
    tables_html_request = tables_html(url=url)
    
    # delay execute so not to trigger ip address block
    time.sleep(random.randrange(7,9))
    
    # webscrape tables/ create dataframes
    
    # table name list with team and year to format
    table_name_li = [
        'team_stats_and_ranking_{}_{}',
        'schedule_and_game_results_{}_{}',
        'team_conversions_{}_{}',
        'passing_{}_{}',
        'rushing_and_receiving_{}_{}',
        'kick_and_punt_returns_{}_{}',
        'kicking_{}_{}',
        'punting_{}_{}',
        'defense_and_fumbles_{}_{}',
        'scoring_summary_{}_{}',
        'touchdown_log_{}_{}',
        'opponent_touchdown_log_{}_{}'
    ]
    
    #file location
    file_loc = 'NFL_Data_{}/{}/'.format(year, team)
    
    table_num = 12
    
    table_df_li = [webscrape_tables(tables=tables_html_request, table_num=i) for i in range(0,table_num)]

    table_df_li = fix_col_labels(table_df_li=table_df_li, team=team)
    
    for t in range(0, table_num):
        
        table_df_li[t].to_csv(file_loc+table_name_li[t].format(team, year))
        
    return  table_df_li
        
        

# Download CSV Files from all teams 2010-2022

In [12]:
team_names=[
    'Cardinals',
    'Falcons',
    'Ravens',
    'Buffalos',
    'Panthers',
    'Bears',
    'Bengals',
    'Browns',
    'Cowboys',
    'Broncos',
    'Lions',
    'Packers',
    'Texans',
    'Colts',
    'Jaguars',
    'Chiefs',
    'Raiders',
    'Chargers',
    'Rams',
    'Dolphins',
    'Vikings',
    'Patriots',
    'Saints',
    'Giants',
    'Jets',
    'Eagles',
    'Steelers',
    '49ers',
    'Seahawks',
    'Buccaneers',
    'Titans',
    'Commanders'
]

In [13]:
years = range(2010,2023)

In [14]:
#this will webscrape all the teams tables from 2010-2022
#do not run 
###                         45-60 min run
#for team in team_names:
#    for year in years:
#        webscrape_create_dataframes_save_csv(team=team, year=year)

# 2023

In [41]:
years = [2023]
years

[2023]

In [ ]:
def webscrape_create_dataframes_save_csv_2023(team: str, year: int):    
    
    team_to_url_df = pd.DataFrame(
        data=[
            [
                'crd', #Cardinals
                'atl', #Falcons
                'rav', #Ravens
                'buf', #Buffalos
                'car', #Panthers
                'chi', #Bears
                'cin', #Bengals
                'cle', #Browns
                'dal', #Cowboys
                'den', #Broncos
                'det', #Lions
                'gnb', #Packers
                'htx', #Texans
                'clt', # Colts
                'jax', #Jaguars
                'kan', #Chiefs
                'rai', #Raiders
                'sdg', #Chargers
                'ram', #Rams
                'mia', #Dolphins
                'min', #Vikings
                'nwe', #Patriots
                'nor', #Saints
                'nyg', #Giants
                'nyj', #Jets
                'phi', #Eagles
                'pit', #Steelers
                'sfo', #49ers
                'sea', #Seahawks
                'tam', #Buccaneers
                'oti', #Titans
                'was' #Commanders
            ]
        ], 
        columns=[
            'Cardinals',
            'Falcons',
            'Ravens',
            'Buffalos',
            'Panthers',
            'Bears',
            'Bengals',
            'Browns',
            'Cowboys',
            'Broncos',
            'Lions',
            'Packers',
            'Texans',
            'Colts',
            'Jaguars',
            'Chiefs',
            'Raiders',
            'Chargers',
            'Rams',
            'Dolphins',
            'Vikings',
            'Patriots',
            'Saints',
            'Giants',
            'Jets',
            'Eagles',
            'Steelers',
            '49ers',
            'Seahawks',
            'Buccaneers',
            'Titans',
            'Commanders'
        ])
    
    # string url
    str_url = 'https://www.pro-football-reference.com/teams/{}/{}.htm'

    #get team url ID
    team_url_str = team_to_url_df[team].values[0]
    
    # URL
    url = str_url.format(team_url_str, year)
    
    # request html text
    tables_html_request = tables_html(url=url)
    
    # delay execute so not to trigger ip address block
    time.sleep(random.randrange(7,9))
    
    # webscrape tables/ create dataframes
    
    # table name list with team and year to format
    table_name_li = [
        'team_stats_and_ranking_{}_{}',
        'schedule_and_game_results_{}_{}',
        'team_conversions_{}_{}',
        'passing_{}_{}',
        'rushing_and_receiving_{}_{}',
        'kick_and_punt_returns_{}_{}',
        'kicking_{}_{}',
        'punting_{}_{}',
        'defense_and_fumbles_{}_{}',
        'scoring_summary_{}_{}',
        'touchdown_log_{}_{}',
        'opponent_touchdown_log_{}_{}'
    ]
    
    #file location
    file_loc = 'NFL_Data_{}/{}/'.format(year, team)
    
    table_num = 12
    
    table_df_li = [webscrape_tables(tables=tables_html_request[1:], table_num=i) for i in range(0,table_num)]

    table_df_li = fix_col_labels(table_df_li=table_df_li, team=team)
    
    for t in range(0, table_num):
        
        table_df_li[t].to_csv(file_loc+table_name_li[t].format(team, year))
        
    return  table_df_li
        
        

In [45]:
for team in team_names:
    for year in years:
        webscrape_create_dataframes_save_csv_2023(team=team, year=year) 

In [ ]:
#sort teams alpha

In [46]:
        data=[
            [
                'crd', #Cardinals
                'atl', #Falcons
                'rav', #Ravens
                'buf', #Buffalos
                'car', #Panthers
                'chi', #Bears
                'cin', #Bengals
                'cle', #Browns
                'dal', #Cowboys
                'den', #Broncos
                'det', #Lions
                'gnb', #Packers
                'htx', #Texans
                'clt', # Colts
                'jax', #Jaguars
                'kan', #Chiefs
                'rai', #Raiders
                'sdg', #Chargers
                'ram', #Rams
                'mia', #Dolphins
                'min', #Vikings
                'nwe', #Patriots
                'nor', #Saints
                'nyg', #Giants
                'nyj', #Jets
                'phi', #Eagles
                'pit', #Steelers
                'sfo', #49ers
                'sea', #Seahawks
                'tam', #Buccaneers
                'oti', #Titans
                'was' #Commanders
            ]
        ], 


In [47]:
data.sort()

AttributeError: 'tuple' object has no attribute 'sort'

In [ ]:
        columns=[
            'Cardinals',
            'Falcons',
            'Ravens',
            'Buffalos',
            'Panthers',
            'Bears',
            'Bengals',
            'Browns',
            'Cowboys',
            'Broncos',
            'Lions',
            'Packers',
            'Texans',
            'Colts',
            'Jaguars',
            'Chiefs',
            'Raiders',
            'Chargers',
            'Rams',
            'Dolphins',
            'Vikings',
            'Patriots',
            'Saints',
            'Giants',
            'Jets',
            'Eagles',
            'Steelers',
            '49ers',
            'Seahawks',
            'Buccaneers',
            'Titans',
            'Commanders'
        ]

In [ ]:
columns.sort()